<a href="https://colab.research.google.com/github/goddoe/hacking-llms-for-low-res-settings/blob/main/prefix_tuning_kor_nli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install evaluate tqdm datasets scikit-learn scipy peft

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 86.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [2]:
import argparse
import os

import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader
from peft import (
    get_peft_config,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    PeftType,
    PrefixTuningConfig,
    PromptEncoderConfig,
)

import evaluate
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, get_linear_schedule_with_warmup, set_seed
from tqdm import tqdm

In [3]:
batch_size = 32
model_name_or_path = "roberta-large"
peft_type = PeftType.PREFIX_TUNING
device = "cuda"

In [4]:
peft_config = PrefixTuningConfig(task_type="SEQ_CLS", num_virtual_tokens=20)
lr = 1e-2

In [5]:
if any(k in model_name_or_path for k in ("gpt", "opt", "bloom")):
    padding_side = "left"
else:
    padding_side = "right"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, padding_side=padding_side)
if getattr(tokenizer, "pad_token_id") is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

In [6]:
train_datasets = load_dataset("kor_nli", "multi_nli")
valid_test_datasets = load_dataset("kor_nli", "xnli")

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Dataset kor_nli downloaded and prepared to /root/.cache/huggingface/datasets/kor_nli/multi_nli/1.0.0/06d9b61bd1372a618df02294965857ff10886d48696f33a32cbea656b71dfcf0. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Generating validation split:   0%|          | 0/2490 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5010 [00:00<?, ? examples/s]

Dataset kor_nli downloaded and prepared to /root/.cache/huggingface/datasets/kor_nli/xnli/1.0.0/06d9b61bd1372a618df02294965857ff10886d48696f33a32cbea656b71dfcf0. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
metric = evaluate.load("accuracy")

In [ ]:
def tokenize_function(examples):
    # max_length=None => use the model max length (it's actually the default)
    outputs = tokenizer(examples["premise"], examples["hypothesis"], truncation=True, max_length=460)
    return outputs


train_tokenized_datasets = train_datasets.map(
    tokenize_function,
    batched=True,
    remove_columns=["premise", "hypothesis"],
)

valid_test_tokenized_datasets = valid_test_datasets.map(
    tokenize_function,
    batched=True,
    remove_columns=["premise", "hypothesis"],
)


# We also rename the 'label' column to 'labels' which is the expected name for labels by the models of the
# transformers library
train_tokenized_datasets = train_tokenized_datasets.rename_column("label", "labels")
valid_test_tokenized_datasets = valid_test_tokenized_datasets.rename_column("label", "labels")

def collate_fn(examples):
    return tokenizer.pad(examples, padding="longest", return_tensors="pt")


# Instantiate dataloaders.
train_dataloader = DataLoader(train_tokenized_datasets['train'], shuffle=True, collate_fn=collate_fn, batch_size=batch_size)
# train_dataloader = DataLoader(train_tokenized_datasets['train'].train_test_split(train_size=1000)['train'], shuffle=True, collate_fn=collate_fn, batch_size=batch_size)
eval_dataloader = DataLoader(
    valid_test_tokenized_datasets["validation"], shuffle=False, collate_fn=collate_fn, batch_size=batch_size
)

Map:   0%|          | 0/392702 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name_or_path, return_dict=True, num_labels=3)

In [ ]:
model

In [ ]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
model

In [ ]:
optimizer = AdamW(params=model.parameters(), lr=lr)


num_epochs = 20

# Instantiate scheduler
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0.06 * (len(train_dataloader) * num_epochs),
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [ ]:
model.to(device)

for epoch in range(num_epochs):
    model.train()
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch.to(device)
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch.to(device)
        with torch.no_grad():
            outputs = model(**batch)
        predictions = outputs.logits.argmax(dim=-1)
        predictions, references = predictions, batch["labels"]
        metric.add_batch(
            predictions=predictions,
            references=references,
        )

    eval_metric = metric.compute()
    print(f"epoch {epoch}:", eval_metric)

In [ ]:
output_path = "outputs/prefix_tuning"
model.save_pretrained("outputs/prefix_tuning")
tokenizer.save_pretrained("outputs/prefix_tuning")

## Load and Inference

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer


config = PeftConfig.from_pretrained(output_path)
inference_model = AutoModelForSequenceClassification.from_pretrained(config.base_model_name_or_path, num_labels=3)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
inference_model = PeftModel.from_pretrained(inference_model, output_path)

inference_model.to(device)
inference_model.eval()
for step, batch in enumerate(tqdm(eval_dataloader)):
    batch.to(device)
    with torch.no_grad():
        outputs = inference_model(**batch)
    predictions = outputs.logits.argmax(dim=-1)
    predictions, references = predictions, batch["labels"]
    metric.add_batch(
        predictions=predictions,
        references=references,
    )

eval_metric = metric.compute()
print(eval_metric)